# Mix the ingredients

In Chapter 2 we read the ingedients/steps, in 3, we parsed those ingredients to gain some understanding on what they do. Now we will take the steps and mix the ingredients. In doing so we will gain a deaper understanding of advanced data structures, like Dictionaries.

In this chapter we read over the steps and operatate on the ingredients.


> read in the steps

In [1]:
import pickle
recipe_as_dict = pickle.load(open("data.pickle", "rb"))

> our program is getting more complex, we will start using the built in Python logging to help us get feedback as we run

In [2]:
import logging as log
from pprint import pformat
logger = log.getLogger()
logger.setLevel(log.DEBUG)

In [3]:
ch = log.StreamHandler()
ch.setFormatter(log.Formatter('%(levelname)s: %(message)s'))
logger.removeHandler(0)
logger.addHandler(ch)
log.info("who loves Pie?")
log.debug("i am a debug message")

INFO: who loves Pie?
DEBUG: i am a debug message


Let's break out the filling and crust steps

In [4]:
#let's make a list of filling 
filling_steps = None
crust_steps = None
for part in recipe_as_dict["Parts"]:
    if 'filling' == part['sub-title']:
        filling_steps = part['steps']
    elif 'crust' == part['sub-title']:
        crust_steps = part['steps']
log.info("{} filling steps and {} crust steps".format(len(filling_steps),
                                                      len(crust_steps)))

INFO: 3 filling steps and 6 crust steps


### let's start with the filling

Inspect the filling_steps

In [5]:
log.info(pformat(filling_steps))
log.info(len(filling_steps))

INFO: ['Line pie plate with unbaked pastry.  Mix together the sugars, flour\n'
 'and spices.  Pour 1/2 of this mixture into the pie plate, smoothing\n'
 'evenly across the bottom.  Lay the apples in the crust, enough so that\n'
 "they reach the top of the plate but aren't mounded over it.  Sprinkle\n"
 'the remaining flour mixture over the top of the apples.  Sprinkle 1\n'
 'tablespoon of lemon juice over the top, then dot with 1 tablespoon of\n'
 'butter.  Add the top crust and cut slits for steam.',
 'Bake 15 minutes at 425 degrees, then reduce heat to 350 and bake\n'
 'another hour.  The trick with apple pie is to be sure and bake it long\n'
 'enough for the apples to be soft.  I usually stick a fork inside one\n'
 'of the slits to make sure, but an hour should be plenty.  If the edge\n'
 'of the crust starts to brown too much, cover it loosely with foil.',
 "It's a good idea to set the pie on a cookie sheet in case the apples\n"
 'are particularly juicy and spill over into the oven

A couple of obsercations:

 * there are three parts
 * there is a bunch of "\n", which mean line break, but the aren't important
 * what may be important is the sentances in each 
 
Let's break this up into a list of list based on sentences.

In [6]:
list_of_list_of_steps = []
for step in filling_steps:
    step = step.replace("\n", " ")
    list_of_list_of_steps.append(step.split(". "))
    log.debug("step")
list_of_list_of_steps

DEBUG: step
DEBUG: step
DEBUG: step


[['Line pie plate with unbaked pastry',
  ' Mix together the sugars, flour and spices',
  ' Pour 1/2 of this mixture into the pie plate, smoothing evenly across the bottom',
  " Lay the apples in the crust, enough so that they reach the top of the plate but aren't mounded over it",
  ' Sprinkle the remaining flour mixture over the top of the apples',
  ' Sprinkle 1 tablespoon of lemon juice over the top, then dot with 1 tablespoon of butter',
  ' Add the top crust and cut slits for steam.'],
 ['Bake 15 minutes at 425 degrees, then reduce heat to 350 and bake another hour',
  ' The trick with apple pie is to be sure and bake it long enough for the apples to be soft',
  ' I usually stick a fork inside one of the slits to make sure, but an hour should be plenty',
  ' If the edge of the crust starts to brown too much, cover it loosely with foil.'],
 ["It's a good idea to set the pie on a cookie sheet in case the apples are particularly juicy and spill over into the oven."]]

We want to identify the keywords that indicate mixing

 * "mix"
 * "pour"
 * "sprinkle"
 * "dot"

In [7]:
known_operations = ("mix", "pour", "sprinkle", "dot")

From Chapter 3, we have a list of ingredients already broken up into quantities and units.

The ingredients will live in three states:

 * The preperation counter (the_counter)
 * The mixing bowl (mixing_bowl)
 * The pie place (pie_plate)
 
We need to move ingredients from state to state from step to step and keep track of the ingredients. Later we will need to interupt the pie-making process by refilling the ingredients on the preperation counter.

> We start out with those steps and identify which ones require action


In [8]:
the_counter = {}
mixing_bowl = {}
pie_plate = {}

We need to break it out into functions:

 * a function to test if it has a mix operation (has_mix_operation)
 * a function to do the mixing (mix)
 
Before we pick it up with the actual steps we prototype each function.


In [9]:
def has_mix_operation(step):
    "test if the step needs a mix operation. Returns: True or False"
    for operation in known_operations:
        if operation in step.lower():
            log.debug("'{}' found in '{}'".format(operation, step))
            return True
    log.warning("'{}' not found in '{}'".format(known_operations, step))
    return False

In [10]:
has_mix_operation("Line pie plate with unbaked pastry")

False

In [11]:
has_mix_operation("Mix together the sugars, flour and spices")

DEBUG: 'mix' found in 'Mix together the sugars, flour and spices'


True

The mix() function is what does the mixing (duh), but it also uses dictionaries (dicts) as input. But first a quick tutorial on dicts.

### Dicts (dictionaries)

A couple notes on dictionaries:

* dicts are not sorted so they don't retain order
* keys must be unique
* dicts point to data values so they retain changes
* the types used for keys and values do not need to be same even inside same dict

Some examples of this...

```python
>>> the_dict = {'a':2, 'b':2, 'c':2, 'd':4, 'e':4}
>>> print(the_dict)
{'d': 4, 'b': 2, 'c': 2, 'a': 2, 'e': 4}
>>> def changeone(passed_dict):
...    passed_dict['d'] = 1337
>>> changeone(the_dict)
>>> print(the_dict)
{'d': 1337, 'b': 2, 'c': 2, 'a': 2, 'e': 4}
```

Now back to our mix() function

In [12]:
from output import LargeItem, IngredientBase, DrySolid, Liquid
from fractions import Fraction

In [13]:
import pickle
shopping_list = pickle.load(open("shopping_list.pickle", "rb"))

In [14]:
shopping_list

[<Ingredient (large item): 3-4 Granny Smith apples, depending on size, peeled and sliced>,
 <Ingredient (solid): 1/2 cup brown sugar>,
 <Ingredient (solid): 1/2 cup granulated sugar>,
 <Ingredient (solid): 1/4 cup flour>,
 <Ingredient (liquid): 1 tsp apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg)>,
 <Ingredient (solid): 2 cups flour>,
 <Ingredient (solid): 1 tsp salt>,
 <Ingredient (solid): 3/4 cup solid shortening (like Crisco)>,
 <Ingredient (liquid): 1/4 to 1/2 cup ice water>]

In [15]:
for x in shopping_list: print(x.__dict__)

{'qty_max': '4', 'unit': 'item', 'original_ingredient_str': '3-4 Granny Smith apples, depending on size, peeled and sliced', 'item': 'Granny Smith apples, depending on size, peeled and sliced', 'qty': Fraction(3, 1)}
{'qty_max': 0, 'unit': 'cup', 'original_ingredient_str': '1/2 cup brown sugar', 'item': 'brown sugar', 'qty': Fraction(1, 2)}
{'qty_max': 0, 'unit': 'cup', 'original_ingredient_str': '1/2 cup granulated sugar', 'item': 'granulated sugar', 'qty': Fraction(1, 2)}
{'qty_max': 0, 'unit': 'cup', 'original_ingredient_str': '1/4 cup flour', 'item': 'flour', 'qty': Fraction(1, 4)}
{'qty_max': 0, 'unit': 'tsp', 'original_ingredient_str': '1 tsp apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg)', 'item': 'apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg)', 'qty': Fraction(1, 1)}
{'qty_max': 0, 'unit': 'cups', 'original_ingredient_str': '2 cups flour', 'item': 'flour', 'qty': Fraction(2, 1)}
{'qty_max': 0, 'unit': 'tsp', 'original_ingredient_str': '1 tsp salt', 'item': 'sal

In [22]:
def move_items(items, from_dict, to_dict, qty=0.0, unit=None, move_all=False):
    """move items from_dict to to_dict, in qty of unit (cups, tbs, tsp) or move_all to move it all.
    """
    for item in items:
        if unit == from_dict['beer'].unit and unit == to_dict['beer'].unit:
            from_dict['beer'].qty -= qty
            to_dict['beer'].qty += qty
            
    
    
start = {"beer": Liquid("4 cups of beer")}
end = {"beer": Liquid("0 cups of beer")}
move_items(["beer", ], start, end, qty=2, unit="cups")
assert start["beer"].qty == Fraction("2")
assert end["beer"].qty == Fraction("2")



In [25]:
the_counter = {}
mixing_bowl = {}
pie_plate = {}

for item in shopping_list:
    the_counter[item.name] = item.qty
    mixing_bowl[item.name] = Fraction("0")
    pie_plate[item.name] = Fraction("0") 

the_counter, mixing_bowl, pie_plate

({'large item': Fraction(3, 1),
  'liquid': Fraction(1, 4),
  'solid': Fraction(3, 4)},
 {'large item': Fraction(0, 1),
  'liquid': Fraction(0, 1),
  'solid': Fraction(0, 1)},
 {'large item': Fraction(0, 1),
  'liquid': Fraction(0, 1),
  'solid': Fraction(0, 1)})

In [ ]:
def mix(the_counter, mixing_bowl, pie_plate, step):
    """Takes the different work areas as dictionaries.
       Inspects the step.
       Adjusts the quanties
       """
    target_items = []
    for item in shopping_list:
        if item.does_match_target(step):
            log.debug("{} matches".format(item))
            target_items.append(item)
            
    if "together" in step:
        move_items(the_counter, mixing_bowl, pie_plate)
    
test_step = "Mix together the sugars, flour and spices"

mix(items, the_counter, mixing_bowl, test_step)


In [ ]:

part_num = 1
for step_part in list_of_list_of_steps:
    print("# part #{}".format(part_num))
    step_num = 1
    for step in step_part:
        print("  - step #{} in part #{}".format(step_num, part_num))
        if has_mix_operation(step):
            mix()
        step_num += 1
    part_num += 1
